# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read .csv file created from WeatherPy
csv_file = pd.read_csv("../WeatherPy/output_data/cities.csv")
csv_file.head()

,City,Lat,Long,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Borba,-4.39,-59.59,80.65,90,100,2.24,BR,1603563472
1,Taldan,53.67,124.80,27.73,94,0,5.39,RU,1603563472
2,Tiksi,71.69,128.87,12.33,91,59,21.47,RU,1603563473
3,Ostrovnoy,68.05,39.51,32.45,97,100,13.38,RU,1603563473
4,Albany,42.60,-73.97,59.00,66,89,12.80,US,1603563467


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [35]:
# Define variables 
locations = csv_file[["Lat", "Long"]]
humidity = csv_file["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# New DataFrame showing cities with ideal weather: 
# A max temperature lower than 85 degrees but higher than 70.
# Wind speed less than 15 mph.
# Zero cloudiness.

ideal_city = csv_file.loc[((csv_file['Max Temp'] <= 85) & (csv_file['Max Temp'] >= 70) \
                           & (csv_file['Wind Speed'] <= 15) & (csv_file['Clouds'] == 0).dropna(how='all'))]

ideal_city

,City,Lat,Long,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
101,Talara,-4.58,-81.27,78.80,74,0,13.87,PE,1603563480
127,Los Andes,-32.83,-70.60,79.00,77,0,3.00,CL,1603563482
166,Kalakamati,-20.59,27.31,80.60,36,0,5.82,BW,1603563485
178,Salalah,17.02,54.09,80.60,42,0,4.70,OM,1603563486
183,Ovalle,-30.60,-71.20,77.20,32,0,10.98,CL,1603563486
184,Saint-Philippe,-21.36,55.77,73.40,68,0,10.29,RE,1603563486
205,Saint-Pierre,-21.34,55.48,71.60,68,0,10.29,RE,1603563487
207,Najrān,17.49,44.13,75.20,17,0,4.70,SA,1603563487
209,Chabahar,25.29,60.64,78.80,54,0,7.02,IR,1603563488
257,Mizdah,31.45,12.98,70.79,24,0,7.25,LY,1603563490


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = ideal_city.loc[:,["City","Country","Lat","Long"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Long,Hotel Name
101,Talara,PE,-4.58,-81.27,
127,Los Andes,CL,-32.83,-70.60,
166,Kalakamati,BW,-20.59,27.31,
178,Salalah,OM,17.02,54.09,
183,Ovalle,CL,-30.60,-71.20,
184,Saint-Philippe,RE,-21.36,55.77,
205,Saint-Pierre,RE,-21.34,55.48,
207,Najrān,SA,17.49,44.13,
209,Chabahar,IR,25.29,60.64,
257,Mizdah,LY,31.45,12.98,


In [7]:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Get latitude, longitude, and city from DataFrame 'ideal_city'
    lat = row['Lat']
    long = row['Long'] 
    city = row['City']
    
    # Set location
    coords = f"{lat}, {long}"
    
    # Set parameters 
    params = {
    "location": coords,
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": g_key}
    
    # Set url and make API request
    print(f"Retrieving Results for Index {index}: {city} at {coords}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Add the hotel name to dataframe
    try:
        print(f"First hotel in {city}  is {results[0]['name']}.") 
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'] 
       
    # If there is no hotel available, show missing field    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  
    print("------------")
    
# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 101: Talara at -4.58, -81.27.
First hotel in Talara  is Casa Andina Standard Talara.
------------
Retrieving Results for Index 127: Los Andes at -32.83, -70.6.
First hotel in Los Andes  is Cardon.
------------
Retrieving Results for Index 166: Kalakamati at -20.59, 27.31.
Missing field/result... skipping.
------------
Retrieving Results for Index 178: Salalah at 17.02, 54.09.
First hotel in Salalah  is Atana Stay Salalah.
------------
Retrieving Results for Index 183: Ovalle at -30.6, -71.2.
First hotel in Ovalle  is Hotel Keo.
------------
Retrieving Results for Index 184: Saint-Philippe at -21.36, 55.77.
First hotel in Saint-Philippe  is Les Embruns Du Baril.
------------
Retrieving Results for Index 205: Saint-Pierre at -21.34, 55.48.
First hotel in Saint-Pierre  is Villa Delisle, Hotel & Spa.
------------
Retrieving Results for Index 207: Najrān at 17.49, 44.13.
Missing field/result... skipping.
------------
Retrieving Results for Index 209: Chabahar at

In [28]:
# Replace missing field in Hotel Name with NaN and drop these
hotel_df[hotel_df == ""] = np.nan
hotel_df = hotel_df.dropna()

# Create New DataFrame
hotel_data = pd.DataFrame(hotel_df)
hotel_data

,City,Country,Lat,Long,Hotel Name
101,Talara,PE,-4.58,-81.27,Casa Andina Standard Talara
127,Los Andes,CL,-32.83,-70.60,Cardon
178,Salalah,OM,17.02,54.09,Atana Stay Salalah
183,Ovalle,CL,-30.60,-71.20,Hotel Keo
184,Saint-Philippe,RE,-21.36,55.77,Les Embruns Du Baril
205,Saint-Pierre,RE,-21.34,55.48,"Villa Delisle, Hotel & Spa"
209,Chabahar,IR,25.29,60.64,Lipar Hotel
257,Mizdah,LY,31.45,12.98,فندق مزدة الكبير
258,Assiut,EG,27.18,31.18,Florence Hotel Assiut
306,Jalu,LY,29.03,21.55,Jalu Hotel


In [32]:
#Save hotel data to a csv file
hotel_data.to_csv("output_data/hotel_data.csv", index=False)

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["Lat", "Long"]]

In [31]:
# Add marker layer on top of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))